In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trip_count_by_zone.sql").getOrCreate()

In [3]:
directory = "C:\\Users\\krlee\\JupyterLab\\fastcam_spark_and_flink\\data"
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [4]:
trip_data = spark.read.csv(f"file:///{directory}/{trip_file}", inferSchema = True, header = True) 
zone_data = spark.read.csv(f"file:///{directory}/{zone_file}", inferSchema = True, header = True) 
trip_data.show(5)
zone_data.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [6]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [8]:
spark.sql("select * from trip_data").show(3)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [9]:
spark.sql("select * from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [11]:
spark.sql("select borough, count(*) from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID) \
group by borough order by count(*) desc").show()

+---------+--------+
|  borough|count(1)|
+---------+--------+
|Manhattan|     923|
|   Queens|      43|
|  Unknown|      17|
| Brooklyn|      12|
|    Bronx|       4|
+---------+--------+



In [12]:
spark.sql("select borough, count(*) from \
(select zone_data.Borough as borough \
from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID) \
group by borough order by count(*) desc").show()

+-------------+--------+
|      borough|count(1)|
+-------------+--------+
|    Manhattan|     825|
|     Brooklyn|      78|
|       Queens|      50|
|        Bronx|      30|
|      Unknown|      15|
|Staten Island|       1|
+-------------+--------+

